<a href="https://colab.research.google.com/github/sidharthdk/BMS-Data-pre-processing-files/blob/main/Ul_Purdue_Battery_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# === UL-PURDUE (UL-PUR) NCA BATTERY DATASET: LOCAL ZIP → CLEANED ZIP ===
# Upload your UL-PUR dataset ZIP to Colab Files panel, then RUN
from google.colab import files
import zipfile
import pandas as pd
import numpy as np
from pathlib import Path

# Step 1: Upload ZIP file
print("📤 Upload your UL-PUR (UL-Purdue) battery dataset ZIP:")
uploaded = files.upload()
zip_path = list(uploaded.keys())[0]
print(f"✅ Processing: {zip_path}")

# Step 2: Extract to temp directory
raw_dir = Path("/content/ul_pur_raw")
raw_dir.mkdir(exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(raw_dir)
print(f"✅ Extracted to {raw_dir}")

csv_files = list(raw_dir.rglob("*.csv"))
print(f"Found {len(csv_files)} CSV files")

# Step 3: UL-PUR config (mostly 18650/pouch NCA cells, typical range 2.7-4.2V)
CONFIG = {
    'interp_gap': 10,           # Reasonable for NCA cycling data
    'v_min': 2.7,               # Common lower cutoff in UL-PUR cycling
    'v_max': 4.2,               # Most common upper limit (some tests 4.0 V)
    'drop_bad_cycles': True,
    'capacity_max': 3.5         # Typical 18650 NCA ~2.8–3.4 Ah; pouch a bit higher
}

clean_dir = Path("/content/ul_pur_cleaned")
clean_dir.mkdir(exist_ok=True)

def clean_ul_pur_file(path: Path) -> bool:
    """Clean single UL-PUR file (cycle summary or timeseries data)"""
    fname = path.name
    print(f" Cleaning {fname}...")
    try:
        df = pd.read_csv(path)
        if df.empty:
            print(" ⚠ Empty file, skipped")
            return False
        orig_rows = len(df)

        # 1) Column standardization (remove spaces, special chars, unify naming)
        df.columns = (
            df.columns.str.strip().str.lower()
            .str.replace(r'[^a-z0-9_]', '', regex=True)
            .str.replace(r'_+', '_', regex=True)
        )

        # Core column mapping (inspired by common BatteryML / UL-PUR style)
        rename_map = {
            'cycle': 'cycle_index', 'cycle_number': 'cycle_index', 'cycle_id': 'cycle_index',
            'test_time': 'test_time_s', 'time_s': 'test_time_s', 'time': 'test_time_s',
            'voltage_v': 'voltage_v', 'voltage': 'voltage_v', 'v': 'voltage_v',
            'current_a': 'current_a', 'current': 'current_a', 'i': 'current_a',
            'temperature_c': 'temperature_c', 'temp': 'temperature_c', 'temperature': 'temperature_c',
            'discharge_capacity_ah': 'discharge_capacity_ah', 'q_d': 'discharge_capacity_ah', 'cap_discharge': 'discharge_capacity_ah',
            'charge_capacity_ah': 'charge_capacity_ah', 'q_c': 'charge_capacity_ah',
            'discharge_energy_wh': 'discharge_energy_wh', 'energy_discharge': 'discharge_energy_wh',
            'charge_energy_wh': 'charge_energy_wh',
            'soc': 'soc', 'state_of_charge': 'soc'
        }
        df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns}, inplace=True)

        # 2) Numeric conversion & basic cleanup
        df = df.apply(pd.to_numeric, errors='coerce')
        df = df.dropna(axis=1, how='all').dropna(how='all')
        df = df.drop_duplicates()

        # 3) Interpolation (group by cycle when possible)
        num_cols = df.select_dtypes(include=np.number).columns
        if len(num_cols) > 0 and len(df) > 1:
            time_col = next((c for c in ['test_time_s', 'time_s', 'time'] if c in df), None)
            cycle_col = next((c for c in ['cycle_index', 'cycle'] if c in df), None)

            if cycle_col and time_col:
                df = df.sort_values([cycle_col, time_col])
                df[num_cols] = df.groupby(cycle_col)[num_cols].transform(
                    lambda x: x.interpolate(method='linear', limit=CONFIG['interp_gap'])
                )
            else:
                if time_col:
                    df = df.sort_values(time_col)
                df[num_cols] = df[num_cols].interpolate(method='linear', limit=CONFIG['interp_gap'])

            # Fill remaining gaps conservatively
            df[num_cols] = df[num_cols].ffill().bfill()
            df = df.dropna(subset=num_cols.intersection(['voltage_v', 'current_a']))

        # 4) Physics-based filtering for NCA cells
        if 'voltage_v' in df:
            df = df[(df['voltage_v'] >= CONFIG['v_min']) & (df['voltage_v'] <= CONFIG['v_max'])]

        # Capacity clipping (avoid unrealistic values)
        cap_cols = [c for c in df.columns if 'capacity_ah' in c or c.startswith('q_')]
        for col in cap_cols:
            if col in df:
                df[col] = df[col].clip(lower=0, upper=CONFIG['capacity_max'])

        # Non-negative energy/power
        for col in num_cols:
            if 'energy' in col or 'power' in col:
                df[col] = df[col].clip(lower=0)

        # 5) Cycle validation — ensure time is monotonic per cycle
        if CONFIG['drop_bad_cycles'] and cycle_col and time_col:
            is_valid = df.groupby(cycle_col)[time_col].apply(
                lambda x: x.is_monotonic_increasing
            )
            bad_cycles = is_valid[~is_valid].index
            if len(bad_cycles) > 0:
                df = df[~df[cycle_col].isin(bad_cycles)]
                print(f" ⚠ Removed {len(bad_cycles)} non-monotonic cycles")

        # 6) Save only if meaningful data remains
        if len(df) < 20:  # slightly higher threshold than SNL example
            print(f" ⚠ Only {len(df)} rows after cleaning — skipped")
            return False

        out_name = f"{path.stem}_cleaned.csv"
        out_path = clean_dir / out_name
        df.to_csv(out_path, index=False)

        retention = len(df) / orig_rows * 100
        print(f" ✅ {len(df):,} rows ({retention:.0f}% retention)")
        return True

    except Exception as e:
        print(f" ❌ {type(e).__name__}: {str(e)[:60]}...")
        return False

# Process all CSV files
print("\n" + "="*70)
processed_files = 0
for i, csv_file in enumerate(csv_files):
    print(f"File {i+1}/{len(csv_files)}: ", end="")
    if clean_ul_pur_file(csv_file):
        processed_files += 1

print(f"\n{'='*70}")
print(f"🎉 UL-PUR NCA CLEANING COMPLETE!")
print(f"✅ Successfully processed: {processed_files}/{len(csv_files)} files")

# Step 7: Create final ZIP of cleaned files
final_zip = "/content/ul_pur_battery_cleaned.zip"
with zipfile.ZipFile(final_zip, 'w', zipfile.ZIP_DEFLATED) as zf:
    for cleaned_csv in clean_dir.glob("*.csv"):
        zf.write(cleaned_csv, cleaned_csv.name)

print(f"\n📦 Download: {final_zip}")
!du -sh {final_zip} {clean_dir}

# Quick validation peek at one cleaned file (if any)
if list(clean_dir.glob("*.csv")):
    sample_path = list(clean_dir.glob("*.csv"))[0]
    sample_df = pd.read_csv(sample_path)
    print(f"\n📊 SAMPLE ({sample_path.name}):")
    print(f"Shape: {sample_df.shape}")
    print(f"Columns: {list(sample_df.columns)}")
    if 'voltage_v' in sample_df:
        print(f"Voltage range: {sample_df['voltage_v'].min():.2f} V → {sample_df['voltage_v'].max():.2f} V")
    print(f"Missing values total: {sample_df.isnull().sum().sum()}")

📤 Upload your UL-PUR (UL-Purdue) battery dataset ZIP:


Saving UL-Purdue-20260122T073731Z-1-001.zip to UL-Purdue-20260122T073731Z-1-001.zip
✅ Processing: UL-Purdue-20260122T073731Z-1-001.zip
✅ Extracted to /content/ul_pur_raw
Found 44 CSV files

File 1/44:  Cleaning UL-PUR_R15-OV3_18650_NCA_23C_2.5-96.5_0.5-0.5C_c_cycle_data.csv...
 ✅ 652 rows (100% retention)
File 2/44:  Cleaning UL-PUR_R10-NA11_18650_NCA_23C_2.5-96.5_0.5-0.5C_k_cycle_data.csv...
 ✅ 455 rows (100% retention)
File 3/44:  Cleaning UL-PUR_R20-NA8_18650_NCA_23C_2.5-96.5_0.5-0.5C_h_timeseries.csv...
 ✅ 102,165 rows (100% retention)
File 4/44:  Cleaning UL-PUR_N10-EX9_18650_NCA_23C_0-100_0.5-0.5C_i_cycle_data.csv...
 ✅ 205 rows (100% retention)
File 5/44:  Cleaning UL-PUR_R20-NA7_18650_NCA_23C_2.5-96.5_0.5-0.5C_g_timeseries.csv...
 ✅ 127,588 rows (100% retention)
File 6/44:  Cleaning UL-PUR_N15-OV3_18650_NCA_23C_0-100_0.5-0.5C_c_timeseries.csv...
 ✅ 44,423 rows (100% retention)
File 7/44:  Cleaning UL-PUR_N10-OV8_18650_NCA_23C_0-100_0.5-0.5C_h_timeseries.csv...
 ✅ 43,618 rows (1